In [22]:
import argparse
import networkx as nx
import torch
import torch.nn as nn

!pip install torch-geometric
from torch_geometric.nn import GCNConv

TODO:
Implement teacher network architecture and training along with flags for datasets and teacher model architecture (Chanikya and Nithin)
eg: python3 train_teacher.py --dataset=cora --model=SAGE --epochs-100 --lr=0.01 . Add flags for other hyperparameters if necessary (Chanikya and Nithin)
Other teacher model architectures - GCN, GAT, APPNP (Chanikya and Nithin + others based on availability)

In [37]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./Cora', name='Cora')

data = dataset[0]
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7


In [46]:
# data.edge_index.t()

tensor([[ 633,    0],
        [1862,    0],
        [2582,    0],
        ...,
        [ 598, 2707],
        [1473, 2707],
        [2706, 2707]])

In [92]:
def load_data(dataset):
    if dataset == "cora":
        dataset = Planetoid(root='./Cora', name='Cora')
        data = dataset[0]
        graph_nx = nx.Graph()
        graph_nx.add_edges_from(data.edge_index.t().tolist())

        # Adding self-loops
        # graph_nx.add_edges_from((n, n) for n in graph_nx.nodes())
        
        # adj_tensor = torch.tensor(nx.to_numpy_array(graph_nx), dtype=torch.float).to('cuda')
        features = data.x
        labels = data.y

        train_idx = data.train_mask.nonzero(as_tuple=True)[0]
        val_idx = data.val_mask.nonzero(as_tuple=True)[0]
        test_idx = data.test_mask.nonzero(as_tuple=True)[0]
        
        return data.edge_index, features, labels, data.train_mask, data.val_mask, data.test_mask

In [93]:
cnt = 0
140+1000+500
2708
for x in data.val_mask:
    cnt += (x==True)
print(cnt)

tensor(500)


GCN => num layers, hidden, input dim, out, dp, activa,  

In [94]:
class GCN(nn.Module):
    def __init__(
        self,
        num_layers,
        input_dim,
        hidden_dim,
        output_dim,
        dropout_ratio,
        activation,
        norm_type="none"
    ):
        super().__init__()
        self.num_layers = num_layers
        self.norm_type = norm_type
        self.dropout = nn.Dropout(dropout_ratio)
        self.activation = activation

        self.layers = nn.ModuleList()
        self.norms = nn.ModuleList()

        if num_layers == 1:
            self.layers.append(GCNConv(input_dim, output_dim))
        else:
            self.layers.append(GCNConv(input_dim, hidden_dim))
            if norm_type == "batch":
                self.norms.append(nn.BatchNorm1d(hidden_dim))
            elif norm_type == "layer":
                self.norms.append(nn.LayerNorm(hidden_dim))

            for _ in range(num_layers - 2):
                self.layers.append(GCNConv(hidden_dim, hidden_dim))
                if norm_type == "batch":
                    self.norms.append(nn.BatchNorm1d(hidden_dim))
                elif norm_type == "layer":
                    self.norms.append(nn.LayerNorm(hidden_dim))

            self.layers.append(GCNConv(hidden_dim, output_dim))

    def forward(self, x, edge_index):
        h_list = []
        h = x
        for l, layer in enumerate(self.layers):
            h = layer(h, edge_index)
            if l != self.num_layers - 1:
                if self.norm_type != "none":
                    h = self.norms[l](h)
                h = self.activation(h)
                h = self.dropout(h)
                h_list.append(h)
        return h_list, h

In [95]:
def train(model, data, edge_index, labels, train_idx, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data, edge_index)[1]
    # print(out[1].shape)
    # print(train_idx)
    loss = criterion(out[train_idx], labels[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()


@torch.no_grad()
def evaluate(model, data, edge_index, labels, idx):
    model.eval()
    out = model(data, edge_index)[1]
    pred = out[idx].argmax(dim=1)
    correct = (pred == labels[idx]).sum().item()
    acc = correct / len(idx)
    return acc

In [99]:
edge_index, features, labels, train_idx, val_idx, test_idx = load_data("cora")
model = GCN(
    num_layers=2,
    input_dim=dataset.num_node_features,
    hidden_dim=64,
    output_dim=dataset.num_classes,
    dropout_ratio=0.8,
    activation=F.relu,
    norm_type="batch"
)


optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(1, 501):
    loss = train(model, features, edge_index, labels, train_idx, optimizer, criterion)
    val_acc = evaluate(model, features, edge_index, labels, val_idx)
    if epoch % 10 == 0 or epoch == 1:
        test_acc = evaluate(model, features, edge_index, labels, test_idx)
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")


Epoch 001 | Loss: 2.4875 | Val Acc: 0.1075 | Test Acc: 0.2264
Epoch 010 | Loss: 0.1396 | Val Acc: 0.1429 | Test Acc: 0.2991
Epoch 020 | Loss: 0.0401 | Val Acc: 0.1396 | Test Acc: 0.2903
Epoch 030 | Loss: 0.0197 | Val Acc: 0.1385 | Test Acc: 0.2891
Epoch 040 | Loss: 0.0257 | Val Acc: 0.1385 | Test Acc: 0.2910
Epoch 050 | Loss: 0.0249 | Val Acc: 0.1385 | Test Acc: 0.2917
Epoch 060 | Loss: 0.0174 | Val Acc: 0.1377 | Test Acc: 0.2925
Epoch 070 | Loss: 0.0235 | Val Acc: 0.1377 | Test Acc: 0.2873
Epoch 080 | Loss: 0.0073 | Val Acc: 0.1374 | Test Acc: 0.2855
Epoch 090 | Loss: 0.0064 | Val Acc: 0.1363 | Test Acc: 0.2855
Epoch 100 | Loss: 0.0139 | Val Acc: 0.1359 | Test Acc: 0.2858
Epoch 110 | Loss: 0.0113 | Val Acc: 0.1363 | Test Acc: 0.2851
Epoch 120 | Loss: 0.0148 | Val Acc: 0.1359 | Test Acc: 0.2884
Epoch 130 | Loss: 0.0074 | Val Acc: 0.1359 | Test Acc: 0.2840
Epoch 140 | Loss: 0.0103 | Val Acc: 0.1359 | Test Acc: 0.2847
Epoch 150 | Loss: 0.0105 | Val Acc: 0.1355 | Test Acc: 0.2814
Epoch 16

In [31]:
model(data.x, data.edge_index)

([tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.8113],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.9004],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0712],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.9754],
          [0.0000, 0.0000, 0.0000,  ..., 2.0819, 0.0000, 0.0000],
          [0.9071, 1.6210, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
         grad_fn=<MulBackward0>),
  tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 2.2638, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 5.3454, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.2454, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.1857, 0.0000]],
         grad_fn=<MulBackward0>)],
 tensor([[ 0.8905,  0.6588, -0.3939,  ...,  0.0129, -1.4998,  0.6112],
         [-0.1973,  0.7312, -2.1733,

In [ ]:
class Teacher:
    def __init__(self, args):
        self.args = args
        pass
    def graph_split():
        pass
    def train_transductive():
        pass
    def train_inductive():
        pass
    

In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Teacher implementation")
    parser.add_argument('--num_runs', type=int, default=1, help='Number of runs')
    parser.add_argument('--setting', type=str, choices=['trans', 'ind'], required=True, help='Setting type: trans or ind')
    parser.add_argument('--data_path', type=str, required=True, help='Path to the dataset')
    parser.add_argument('--model_name', type=str, default='SAGE', help='Name of the model(SAGE, GCN, GAT, APPNP)')
    parser.add_argument('--num_layers', type=int, default=2, help='Number of layers in the model')
    parser.add_argument('--hidden_dim', type=int, default=128, help='Hidden dimension size')
    parser.add_argument('--drop_out', type=float, default=0, help='Dropout rate')
    parser.add_argument('--batch_sz', type=int, default=512, help='Batch size')
    parser.add_argument('--learning_rate', type=float, default=0.01, help='Learning rate')
    parser.add_argument('--output_path', type=str, default='./output', help='Path to save output')
    
    args = parser.parse_args()

    